In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

#variable
SUMMARY = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv'
TSS = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Merged_Causality_TSS_TSS.tab'
ANNOT = '/storage/resources/dbase/human/hg19/gencode_gene_annotations_hg19.csv'
sep='"\t"'
command = "cat %s |awk -F'\t' '{print $1%s$2%s$75%s$4}' > all_analyzed_strs.tab"%(SUMMARY, sep,sep,sep)
value = os.system(command)

In [20]:
#within dist_tss=1500bps of TSS or TES  (TS)
dist_ts = 1500

annot = pd.read_csv(ANNOT)
annot['attss'] = annot['gene.start'].apply(lambda x: x-dist_ts)
annot['attes'] = annot['gene.stop'].apply(lambda x: x+dist_ts)

#tss
tss = annot[['gene.chr', 'attss', 'gene.start', 'gene.id']].copy()
tss.columns=['chrom','start', 'stop','gene']
tss.to_csv('dist_tss.tab', index=None, sep='\t')
command = "bedtools intersect -a dist_tss.tab -b all_analyzed_strs.tab -wo > intersect_tss.tab"
value = os.system(command)
inter_tss = pd.read_csv('intersect_tss.tab', sep='\t', header=None)
inter_tss = inter_tss.loc[inter_tss[3]==inter_tss[7]][[4,5,6,7]]
inter_tss.columns=['chrom','str.start', 'str.end','gene']

estrs=pd.read_csv(SUMMARY, sep='\t'); print (len(set(list(estrs['chrom']+estrs['str.id']))), '\t',len(set(list(estrs['gene']))))
estrs=estrs.loc[estrs['E.tissues']>=1] ; print (len(set(list(estrs['chrom']+estrs['str.id']))), '\t',len(set(list(estrs['gene']))))
print(estrs.shape, inter_tss.shape)

I = inter_tss.merge(estrs, on=['chrom','str.start', 'str.end','gene'], how='inner')
print (len(set(list(I['chrom']+I['str.id']))), '\t',len(set(list(I['gene']))))
I.shape

125758 	 17517
20192 	 11869
(25619, 77) (1679, 4)
303 	 299


(303, 77)

In [21]:
#tes
tes = annot[['gene.chr', 'gene.stop', 'attes', 'gene.id']].copy()
tes.columns=['chrom','start', 'stop','gene']
tes.to_csv('dist_tes.tab', index=None, sep='\t')
command = "bedtools intersect -a dist_tes.tab -b all_analyzed_strs.tab -wo > intersect_tes.tab"
value = os.system(command)

inter_tes = pd.read_csv('intersect_tes.tab', sep='\t', header=None)
inter_tes = inter_tes.loc[inter_tes[3]==inter_tes[7]][[4,5,6,7]]
inter_tes.columns=['chrom','str.start', 'str.end','gene']

estrs=pd.read_csv(SUMMARY, sep='\t'); print (len(set(list(estrs['chrom']+estrs['str.id']))), '\t',len(set(list(estrs['gene']))))
estrs=estrs.loc[estrs['E.tissues']>=1] ; print (len(set(list(estrs['chrom']+estrs['str.id']))), '\t',len(set(list(estrs['gene']))))
print(estrs.shape, inter_tes.shape)

I = inter_tes.merge(estrs, on=['chrom','str.start', 'str.end','gene'], how='inner')
print (len(set(list(I['chrom']+I['str.id']))), '\t',len(set(list(I['gene']))))
I.shape

125758 	 17517
20192 	 11869
(25619, 77) (1714, 4)
278 	 274


(278, 77)

In [ ]:
#TSS =========================================================
         TSS  	non TSS 	 Total STR
estrs 	 303 	19889   	 20192
nonestr	 1376	104190  	105566
Total	 1679	124079  	125758

Chi-square with Yates correction
  Chi squared equals 4.852 with 1 degrees of freedom. 
  The two-tailed P value equals 0.0276

#TES =========================================================
        TES  	 nonTES 	 Total STR
estrs 	 278	 19914  	20192
non-est	 1436	 104130 	105566
Total	 1714	 124044 	125758

Chi-square with Yates correction
  Chi squared equals 0.023 with 1 degrees of freedom. 
  The two-tailed P value equals 0.8791
